In [32]:
%load_ext autoreload
import os
from tqdm import tqdm

from modules.constants import *
from modules.utilities import *
from modules.text_classifiers import *

random_sate = 5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
columns = ["title", "body"]
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])

In [6]:
issue_df = read_aidev(FileName.POP_ISSUE)
issue_df

,id,number,title,body,user,state,created_at,closed_at,html_url
0,3082498013,1542,Accept api_key in from_provider,**Is your feature request related to a problem...,jeroenvds,closed,2025-05-22T08:22:22Z,2025-07-17T22:34:21Z,https://github.com/567-labs/instructor/issues/...
1,3164229012,94,Workflows Coming Soon - tools reimagined,### Project Version\n\n5.5.0\n\n### Bug Descri...,guidedways,closed,2025-06-20T19:42:16Z,2025-06-20T20:08:12Z,https://github.com/BeehiveInnovations/zen-mcp-...
2,3198054361,107,[bug]PromptX工具沙箱缓存机制不支持工具集更新,"当工具集需要改进的时候, 会遇到改进后代码和沙箱中的代码不一致的情况.对鲁班自己调试代码造成...",simonfishgit,closed,2025-07-03T05:20:38Z,2025-07-10T01:44:56Z,https://github.com/Deepractice/PromptX/issues/107
3,3208181218,129,架构优化：统一项目路径管理体系 - 合并DirectoryService与ProjectMa...,# 🎯 架构优化：统一项目路径管理体系\n\n## 🔍 问题描述\n\n当前PromptX在...,deepracticexs,closed,2025-07-07T09:12:24Z,2025-07-07T11:05:45Z,https://github.com/Deepractice/PromptX/issues/129
4,2975033421,836,Broken links to notebooks in documentation,On [this documentation page](https://incatools...,justaddcoffee,closed,2025-04-06T15:46:33Z,2025-06-05T23:32:27Z,https://github.com/INCATools/ontology-access-k...
...,...,...,...,...,...,...,...,...,...
4609,2722795232,948,[Node.jsのバージョンをv22に上げる] いいねボタンを作ろう,## 何をどうしたいか？\n\n現状、Node.jsがv20やそれ以下を前提としているチュー...,suin,closed,2024-12-06T11:28:48Z,2025-05-24T09:23:17Z,https://github.com/yytypescript/book/issues/948
4610,2722796490,949,[Node.jsのバージョンをv22に上げる] Jestでテストを書こう,## 何をどうしたいか？\n\n現状、Node.jsがv20やそれ以下を前提としているチュー...,suin,closed,2024-12-06T11:29:12Z,2025-05-24T12:42:13Z,https://github.com/yytypescript/book/issues/949
4611,2722797708,950,[Node.jsのバージョンをv22に上げる] ESLintでコードを検証しよう,## 何をどうしたいか？\n\n現状、Node.jsがv20やそれ以下を前提としているチュー...,suin,closed,2024-12-06T11:29:33Z,2025-05-25T08:37:05Z,https://github.com/yytypescript/book/issues/950
4612,3032958979,976,JavaScriptにも静的メソッドの構文（機能）がある,> JavaScriptにはJavaのような静的メソッドの機能がありません。代わりに、クラス...,ssssota,closed,2025-05-01T01:43:01Z,2025-05-23T11:42:55Z,https://github.com/yytypescript/book/issues/976


In [25]:
title = "Accept api_key in from_provider"
body = """**Is your feature request related to a problem? Please describe.**
It would be helpful if you could provide the API key as part of the from_provider constructor, as settings are often managed outside the environment variables. Currently the only option is to pass it to the client, but that removes the LLM provider-agnostic solution of from_provider()

**Describe the solution you'd like**
This code should be accepted
```
    instructor_client = instructor.from_provider(
        model=settings.LLM_MODEL,
        api_key=settings.LLM_API_KEY
    )
```

Today this results in an Exception:
`The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable`

**Describe alternatives you've considered**
- Environment variables: Not easy to change at runtime etc.
- Setting the API key directly on the client: Impossible to write LLM provider-agnostic code

**Additional context**
n/a
"""

preprocess = None

if preprocess is not None:
    title = preprocess(title)
    body = preprocess(body)

text = title + "\n" + body


In [47]:
def filter_performance_issues(filename, columns,preprocessor, classifier):
    df = read_aidev(filename)
    output_rows = []

    for _, row in tqdm(df.iterrows()):
        text = ""
        for col in columns:
            if isinstance(row[col], str):
                if text != "":
                    text += "\n"
                
                val = row[col]
                if preprocessor is not None:
                    val = preprocessor.preprocess(val)

                text += val

        if text and text != "" and classifier.classify(text):
            output_rows.append(row)
            row["matched_words"] = classifier.get_matches()

    output_df = pd.DataFrame(output_rows)
    
    return output_df

In [48]:
filename = FileName.POP_ISSUE
columns = ["title", "body"]
keywords = set(["performance", "compile-time-hog", "perf", "hang", "optimization", "responsive", "slow" , "speed", "latency", "throughput", "wait", "slow", "fast", "lag", "tim", "minor", "stuck", "instant", "respons", "react", "speed", "latenc", "perform", "throughput", "hang", "memory", "leak"])

preprocessor = None
classifier = RegexClassifier(keywords=keywords)

filter_performance_issues(filename, columns, preprocessor, classifier)


4614it [00:01, 2745.26it/s]


,id,number,title,body,user,state,created_at,closed_at,html_url,matched_words
1,3164229012,94,Workflows Coming Soon - tools reimagined,### Project Version\n\n5.5.0\n\n### Bug Descri...,guidedways,closed,2025-06-20T19:42:16Z,2025-06-20T20:08:12Z,https://github.com/BeehiveInnovations/zen-mcp-...,[performs]
3,3208181218,129,架构优化：统一项目路径管理体系 - 合并DirectoryService与ProjectMa...,# 🎯 架构优化：统一项目路径管理体系\n\n## 🔍 问题描述\n\n当前PromptX在...,deepracticexs,closed,2025-07-07T09:12:24Z,2025-07-07T11:05:45Z,https://github.com/Deepractice/PromptX/issues/129,[optimization]
6,3200531167,4301,`Pkg.instantiate` shouldn't need to uncompress...,(This is not specific to `Pkg.instantiate` rea...,KristofferC,closed,2025-07-03T20:15:07Z,2025-07-04T08:34:05Z,https://github.com/JuliaLang/Pkg.jl/issues/4301,"[instantiate, fast, instantiate]"
8,3244979480,1018,Implement context engineering,Implement below in src/praisonai-agents/praiso...,MervinPraison,closed,2025-07-19T06:14:41Z,2025-07-19T08:45:52Z,https://github.com/MervinPraison/PraisonAI/iss...,"[Performance, perform, responsibility, FastAPI..."
10,3160777773,986,feat: Implement local-first chat persistence w...,## Summary\n\nImplement local-first chat persi...,AtlantisPleb,closed,2025-06-19T15:52:01Z,2025-06-20T01:21:19Z,https://github.com/OpenAgentsInc/openagents/is...,"[Performance, reactive, time, timestamps, time..."
...,...,...,...,...,...,...,...,...,...,...
4602,2372703051,8599,Package scoping fails when workspace glob has ...,### Verify canary release\r\n\r\n- [X] I verif...,timostamm,open,2024-06-25T13:26:28Z,None,https://github.com/vercel/turborepo/issues/8599,"[timostamm, timostamm]"
4603,3028076678,10403,`crates/turborepo-scm/src/git.rs` panics in Gi...,### Verify canary release\n\n- [ ] I verified ...,ScarletFlash,open,2025-04-29T12:07:43Z,None,https://github.com/vercel/turborepo/issues/10403,"[memory, slower]"
4605,2911626727,1906,Replace react-scripts with Vite,None,whitphx,closed,2025-03-11T19:16:18Z,2025-03-12T09:02:41Z,https://github.com/whitphx/streamlit-webrtc/is...,[react]
4607,2327353475,3110,Picker prop 'migrate' warning despite not usin...,<!--\r\nNOTE: please submit only bug reports h...,king960,closed,2024-05-31T09:21:35Z,2025-02-25T00:34:53Z,https://github.com/wix/react-native-ui-lib/iss...,"[react, react, React, React, React]"
